In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Clean data for modeling

pitch = pd.read_csv('../data/mlb-pitches.csv', index_col = [0])
pitch = pitch[['player_name', 'p_throws', 'pitch_type','release_speed', 'release_spin_rate', 'spin_axis', 
               'pfx_-x', 'pfx_z', 'bauer_units', 'effective_speed', 'release_pos_-x', 'release_pos_x', 
               'release_pos_z', 'release_extension', 'release_pos_y', 'plate_-x', 'plate_x', 'plate_z', 
               'type', 'balls','strikes', 'pitch_count', 'delta_run_exp', 'stand', 'bb_type', 'description', 
               'events', 'hit_distance_sc', 'launch_speed', 'launch_angle', 'launch_speed_angle', 'woba_value', 
               'woba_denom', 'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle', 'babip_value', 
               'iso_value', 'at_bat_number', 'pitch_number', 'inning', 'inning_topbot', 'home_score', 
               'away_score', 'post_home_score', 'post_away_score', 'on_1b', 'on_2b', 'on_3b', 
               'outs_when_up']].copy()

pitch.dropna(subset = ['pitch_type', 'release_speed', 'release_spin_rate', 'pfx_-x', 
                       'release_extension', 'delta_run_exp'], inplace = True)

#Rename some columns
col_dict = {
    'release_speed': 'velo',
    'release_spin_rate': 'spin_rate',
    'launch_speed': 'exit_velo',
    'estimated_ba_using_speedangle': 'xba',
    'estimated_woba_using_speedangle': 'xwobacon'
}
pitch.rename(columns = col_dict, inplace = True)
pitch.to_csv('../data/model-data.csv')

In [3]:
# Run Expectany Table

# 2010-2015 Run Expectancy
matrix = [[0, 0, 0, 0, 0.481], [1, 0, 0, 0, 0.859], [0, 1, 0, 0, 1.100], [1, 1, 0, 0, 1.437], 
          [0, 0, 1, 0, 1.350], [1, 0, 1, 0, 1.784], [0, 1, 1, 0, 1.964], [1, 1, 1, 0, 2.292], 
          [0, 0, 0, 1, 0.254], [1, 0, 0, 1, 0.509], [0, 1, 0, 1, 0.664], [1, 1, 0, 1, 0.884], 
          [0, 0, 1, 1, 0.950], [1, 0, 1, 1, 1.130], [0, 1, 1, 1, 1.376], [1, 1, 1, 1, 1.541],
          [0, 0, 0, 2, 0.098], [1, 0, 0, 2, 0.224], [0, 1, 0, 2, 0.319], [1, 1, 0, 2, 0.429], 
          [0, 0, 1, 2, 0.353], [1, 0, 1, 2, 0.478], [0, 1, 1, 2, 0.580], [1, 1, 1, 2, 0.752]]

# 2019 Run Expectancy
#matrix = [[0, 0, 0, 0, 0.544], [1, 0, 0, 0, 0.935], [0, 1, 0, 0, 1.147], [1, 1, 0, 0, 1.537], 
#          [0, 0, 1, 0, 1.369], [1, 0, 1, 0, 1.759], [0, 1, 1, 0, 1.971], [1, 1, 1, 0, 2.362], 
#          [0, 0, 0, 1, 0.298], [1, 0, 0, 1, 0.564], [0, 1, 0, 1, 0.713], [1, 1, 0, 1, 0.979], 
#          [0, 0, 1, 1, 0.953], [1, 0, 1, 1, 1.219], [0, 1, 1, 1, 1.368], [1, 1, 1, 1, 1.634],
#          [0, 0, 0, 2, 0.115], [1, 0, 0, 2, 0.242], [0, 1, 0, 2, 0.339], [1, 1, 0, 2, 0.467], 
#          [0, 0, 1, 2, 0.391], [1, 0, 1, 2, 0.518], [0, 1, 1, 2, 0.615], [1, 1, 1, 2, 0.743]]

re = pd.DataFrame(matrix, columns = ['on_1b', 'on_2b', 'on_3b', 'outs_when_up', 're'])
re.to_csv('../data/run_expectancy_table.csv')

In [4]:
# wOBA by Count

woba_value = [['0-0', 0.310, 0.000, 1, 0, -0.037], ['0-0', 0.310, 0.000, 0, 1, 0.035], 
              ['0-1', 0.262, -0.037, 1, 0, -0.051], ['0-1', 0.262, -0.037, 0, 1, 0.024],
              ['0-2', 0.196, -0.087, 1, 0, -0.150], ['0-2', 0.196, -0.087, 0, 1, 0.021],
              ['1-0', 0.355, 0.035, 1, 0, -0.048], ['1-0', 0.355, 0.035, 0, 1, 0.062],
              ['1-1', 0.293, -0.013, 1, 0, -0.054], ['1-1', 0.293, -0.013, 0, 1, 0.045],
              ['1-2', 0.223, -0.067, 1, 0, -0.171], ['1-2', 0.223, -0.067, 0, 1, 0.038],
              ['2-0', 0.436, 0.097, 1, 0, -0.062], ['2-0', 0.436, 0.097, 0, 1, 0.143],
              ['2-1', 0.352, 0.032, 1, 0, -0.061], ['2-1', 0.352, 0.032, 0, 1, 0.090],
              ['2-2', 0.273, -0.028, 1, 0, -0.209], ['2-2', 0.273, -0.028, 0, 1, 0.085],
              ['3-0', 0.622, 0.239, 1, 0, -0.177], ['3-0', 0.622, 0.239, 0, 1, 0.051],
              ['3-1', 0.470, 0.123, 1, 0, -0.066], ['3-1', 0.470, 0.123, 0, 1, 0.168],
              ['3-2', 0.384, 0.057, 1, 0, -0.294], ['3-2', 0.384, 0.057, 0, 1, 0.234]]

woba_value = pd.DataFrame(woba_value, columns = ['pitch_count', 'count_woba_value', 'wraa',
                                                 'is_strike', 'is_ball', 'wraa_change'])
woba_value.head()
#woba_value.to_csv('../data/count_woba_value.csv')

#count_woba = pd.read_csv('../data/count_woba_value.csv', index_col = [0])
#data = pd.merge(data, woba_value, how = 'left', on = ['pitch_count'])

#data['count_woba_diff'] = round(-data['count_woba_value'].diff(1), 3)
#data['count_woba_after'] = data['count_woba_value'] + data['count_woba_diff']
#data['count_woba_diff'].fillna(0, inplace = True)
#data['count_woba_after'].fillna(0, inplace = True)
##data['events'].fillna('na', inplace = True)
##data['count_woba_after'] = [0 if x != 'na' else y for (x, y) in zip(data['events'], data['count_woba_after'])]
#data['count_woba_diff'] = data['count_woba_after'] - data['count_woba_value']
#data['xRV'] = round((data['count_woba_diff']) / 1.15, 3)

,pitch_count,count_woba_value,wraa,is_strike,is_ball,wraa_change
0,0-0,0.310,0.000,1,0,-0.037
1,0-0,0.310,0.000,0,1,0.035
2,0-1,0.262,-0.037,1,0,-0.051
3,0-1,0.262,-0.037,0,1,0.024
4,0-2,0.196,-0.087,1,0,-0.150


In [5]:
woba = pd.read_csv('../data/count-rv.csv')
woba.drop(columns = ['Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 
                     'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 
                     'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21','Unnamed: 22', 
                     'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25'], inplace = True)
print(woba.shape)
woba.head()

(24, 5)


,pitch_count,woba,is_strike,is_ball,rv
0,0-0,0.314,1,0,-0.036
1,0-0,0.314,0,1,0.028
2,0-1,0.262,1,0,-0.045
3,0-1,0.262,0,1,0.021
4,0-2,0.196,1,0,-0.134


In [6]:
# Add RV to model-data.csv

data = pd.read_csv('../data/model-data.csv', index_col = [0])

data = data[data['events'] != 'wild_pitch']
data = data[data['events'] != 'passed_ball']
data = data[data['events'] != 'stolen_base_2b']

data['events'].replace(['sac_bunt', 'double_play', 'caught_stealing_2b', 'strikeout_double_play',
                        'other_out', 'sac_fly_double_play', 'pickoff_2b', 'pickoff_3b', 'triple_play'
                        'caught_stealing_home', 'pickoff_caught_stealing_2b', 'pickoff_3b', 
                        'sac_bunt_double_play', 'pickoff_caught_stealing_3b', 'pickoff_1b', 
                        'caught_stealing_3b', 'triple_play', 'caught_stealing_home', 
                        'strikeout'], 'field_out', inplace = True)
data['events'].replace(['catcher_interf'], 'field_error', inplace = True)

data['description'].replace(['called_strike', 'swinging_strike', 'swinging_strike_blocked', 'missed_bunt',
                             'foul'], 'strike', inplace = True)
data['description'].replace(['passed_ball', 'wild_pitch'], 'ball', inplace = True)

data['is_strike'] = [1 if x != 'B' else 0 for x in data['type']]
#data['is_strike'] = 
data['is_ball'] = [1 if x == 'B' else 0 for x in data['type']]

data['inning_topbot'] = data.inning_topbot.map({'Top': 0, 'Bot': 1})
data['on_1b'] = [1 if x > 1 else 0 for x in data['on_1b']]
data['on_2b'] = [1 if x > 1 else 0 for x in data['on_2b']]
data['on_3b'] = [1 if x > 1 else 0 for x in data['on_3b']]

data['final_pitch_ab'] = [1 if x == x else 0 for x in data['events']]
data['out_to_end_inning'] = [1 if x == 'field_out' and y == 2 else 0 for (x, y) 
                             in zip(data['events'], data['outs_when_up'])]
data['home_runs'] = data['post_home_score'] - data['home_score']
data['away_runs'] = data['post_away_score'] - data['away_score']
data['runs'] = data['home_runs'] + data['away_runs']

#re = pd.read_csv('../data/run_expectancy_table.csv', index_col = [0])
data = pd.merge(data, re, how = 'left', on = ['on_1b', 'on_2b', 'on_3b', 'outs_when_up'])

data['re_change'] = round(-data['re'].diff(1), 3)
data['re_change'].fillna(-0.098, inplace = True)
data['re_end_state'] = data['re'] + data['re_change']
data['re_end_state'] = [0 if x == 1 else y for (x, y) in zip(data['out_to_end_inning'], data['re_end_state'])]
data['re_change'] = [-y if x == 1 else z for (x, y, z) in zip(data['out_to_end_inning'], 
                                                              data['re'], data['re_change'])]
data['re_change'].replace([-0.000], 0.000, inplace = True)
data['re24'] = data['re_change'] + data['runs']
#mlb_lw = data.groupby(['events', 'on_1b', 'on_2b', 'on_3b', 'outs_when_up'], as_index = False)['re24'].mean()
mlb_lw = data.groupby(['events'], as_index = False)['re24'].mean()
lw_ball_in_play = pd.DataFrame(mlb_lw)
lw_ball_in_play['re24'] = round(lw_ball_in_play['re24'], 3)
lw_ball_in_play.rename(columns = {'re24': 'lin_weight_above_avg'}, inplace = True)
# for lin weights based on base out state 
#data = pd.merge(data, lw, how = 'left', on = ['events', 'on_1b', 'on_2b', 'on_3b', 'outs_when_up'])
data = pd.merge(data, lw_ball_in_play, how = 'left', on = ['events'])
data['lin_weight_above_outs'] = data['lin_weight_above_avg'] + 0.250
data['woba_scale'] = 1.209
data['lin_weights_rel_outs'] = round(data['lin_weight_above_avg'] / data['woba_scale'], 3)
data['lin_weights_not_rel_outs'] = round(data['lin_weight_above_outs'] / data['woba_scale'], 3)
#data['lin_weight'].fillna(0, inplace = True)

#count_woba = pd.read_csv('../data/count_woba_value.csv', index_col = [0])
data = pd.merge(data, woba, how = 'left', on = ['pitch_count', 'is_strike', 'is_ball'])
#data['wraa_after'] = data['wraa'] + data['wraa_change'] 
#data['wraa_re'] = data['wraa_change'] + data['runs']
#mlb_lw_bs = data.groupby(['pitch_count'], as_index = False)['wraa_re'].mean()
#lw_bs = pd.DataFrame(mlb_lw_bs)
#lw_bs['wraa_re'] = round(lw_bs['wraa_re'], 3)
#lw_bs.rename(columns = {'wraa_re': 'bs_lin_weight_above_avg'}, inplace = True)
#data = pd.merge(data, lw_bs, how = 'left', on = ['pitch_count'])
data['lin_weights_rel_outs'].fillna(0, inplace = True)
#data['rv'] = data['lin_weights_rel_outs'] + data['wraa_re']
##data['rv'] = data['lin_weights_rel_outs'] + data['bs_lin_weight_above_avg']
data['rv2'] = ((data['lin_weights_rel_outs'] + data['rv']) * 100)

data.to_csv('../data/model-pitches-rv.csv')

pd.set_option('max_columns', None)
print(data.shape)
data.head(10)

(705396, 68)


,player_name,p_throws,pitch_type,velo,spin_rate,spin_axis,pfx_-x,pfx_z,bauer_units,effective_speed,release_pos_-x,release_pos_x,release_pos_z,release_extension,release_pos_y,plate_-x,plate_x,plate_z,type,balls,strikes,pitch_count,delta_run_exp,stand,bb_type,description,events,hit_distance_sc,exit_velo,launch_angle,launch_speed_angle,woba_value,woba_denom,xba,xwobacon,babip_value,iso_value,at_bat_number,pitch_number,inning,inning_topbot,home_score,away_score,post_home_score,post_away_score,on_1b,on_2b,on_3b,outs_when_up,is_strike,is_ball,final_pitch_ab,out_to_end_inning,home_runs,away_runs,runs,re,re_change,re_end_state,re24,lin_weight_above_avg,lin_weight_above_outs,woba_scale,lin_weights_rel_outs,lin_weights_not_rel_outs,woba,rv,rv2
0,"Smith, Will",L,FF,92.3,2330.0,148.0,-8.28,16.56,25.243770,92.8,-1.40,1.40,6.80,6.5,54.03,0.69,-0.69,2.83,X,1,2,1-2,-0.073,R,ground_ball,hit_into_play,field_out,13.0,95.2,-13.0,2.0,0.0,1.0,0.174,0.158,0.0,0.0,61,4,9,0,5,0,5,0,0,0,0,2,1,0,1,1,0,0,0,0.098,-0.098,0.000,-0.098,-0.25,0.0,1.209,-0.207,0.0,0.223,-0.153,-36.0
1,"Smith, Will",L,SL,80.6,2254.0,315.0,9.24,5.76,27.965261,81.2,-1.60,1.60,6.64,6.4,54.15,0.71,-0.71,2.62,S,1,1,1-1,-0.027,R,NaN,strike,NaN,108.0,75.3,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,3,9,0,5,0,5,0,0,0,0,2,1,0,0,0,0,0,0,0.098,0.000,0.098,0.000,NaN,NaN,1.209,0.000,NaN,0.293,-0.048,-4.8
2,"Smith, Will",L,CU,75.5,1940.0,328.0,7.80,-6.12,25.695364,75.2,-1.46,1.46,6.88,6.2,54.34,0.04,-0.04,2.46,S,1,0,1-0,-0.020,R,NaN,strike,NaN,157.0,83.5,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,2,9,0,5,0,5,0,0,0,0,2,1,0,0,0,0,0,0,0.098,0.000,0.098,0.000,NaN,NaN,1.209,0.000,NaN,0.355,-0.042,-4.2
3,"Smith, Will",L,CU,75.0,2017.0,330.0,8.28,-8.28,26.893333,74.5,-1.53,1.53,6.83,5.9,54.61,-2.10,2.10,3.89,B,0,0,0-0,0.016,R,NaN,ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,1,9,0,5,0,5,0,0,0,0,2,0,1,0,0,0,0,0,0.098,0.000,0.098,0.000,NaN,NaN,1.209,0.000,NaN,0.314,0.028,2.8
4,"Smith, Will",L,FF,91.2,2281.0,143.0,-7.56,15.36,25.010965,90.9,-1.49,1.49,6.66,6.3,54.15,0.31,-0.31,2.80,X,1,0,1-0,-0.189,L,ground_ball,hit_into_play,field_out,9.0,93.3,-18.0,2.0,0.0,1.0,0.100,0.090,0.0,0.0,60,2,9,0,5,0,5,0,0,0,0,1,1,0,1,0,0,0,0,0.254,-0.156,0.098,-0.156,-0.25,0.0,1.209,-0.207,0.0,0.355,-0.042,-24.9
5,"Smith, Will",L,FF,92.5,2319.0,145.0,-6.48,17.40,25.070270,92.0,-1.35,1.35,6.74,6.3,54.19,1.04,-1.04,3.58,B,0,0,0-0,0.026,L,NaN,ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,1,9,0,5,0,5,0,0,0,0,1,0,1,0,0,0,0,0,0.254,0.000,0.254,0.000,NaN,NaN,1.209,0.000,NaN,0.314,0.028,2.8
6,"Smith, Will",L,SL,80.3,2328.0,304.0,3.96,2.16,28.991283,80.8,-1.81,1.81,6.55,6.2,54.30,0.69,-0.69,0.83,S,0,2,0-2,-0.152,L,NaN,strike,field_out,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,0.0,0.0,59,3,9,0,5,0,5,0,0,0,0,0,1,0,1,0,0,0,0,0.481,-0.227,0.254,-0.227,-0.25,0.0,1.209,-0.207,0.0,0.196,-0.134,-34.1
7,"Smith, Will",L,SL,79.5,2368.0,306.0,6.00,4.56,29.786164,79.4,-1.72,1.72,6.56,6.1,54.44,-0.12,0.12,1.99,S,0,1,0-1,-0.052,L,NaN,strike,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59,2,9,0,5,0,5,0,0,0,0,0,1,0,0,0,0,0,0,0.481,0.000,0.481,0.000,NaN,NaN,1.209,0.000,NaN,0.262,-0.045,-4.5
8,"Smith, Will",L,FF,91.8,2299.0,146.0,-8.88,17.64,25.043573,91.8,-1.39,1.39,6.75,6.5,54.04,0.72,-0.72,2.77,S,0,0,0-0,-0.038,L,NaN,strike,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59,1,9,0,5,0,5,0,0,0,0,0,1,0,0,0,0,0,0,0.481,0.000,0.481,0.000,NaN,NaN,1.209,0.000,NaN,0.314,-0.036,-3.6
9,"Gsellman, Robert",R,SI,94.3,1982.0,221.0,16.20,10.56,21.018028,94.4,1.20,-1.20,6.05,6.5,53.98,0.26,-0.26,1.78,X,0,2,0-2,-0.061,R,popup,hit_into_play,field_out,140.0,75.3,65.0,3.0,0.0,1.0,0.000,0.000,0.0,0.0,58,3,8,1,5,0,5,0,0,0,0,2,1,0,1,1,0,0,0,0.098,-0.098,0.000,-0.098,-0.25,0.0,1.209,-0.207,0.0,0.196,-0.134,-34.1


In [7]:
features = data[['velo', 'spin_rate', 'pfx_-x', 'pfx_z', 'release_extension', 'release_pos_x', 
                 'release_pos_z', 'plate_x', 'plate_z', 're24', 'lin_weight_above_avg', 
                 'lin_weight_above_outs', 'lin_weights_rel_outs', 'lin_weights_not_rel_outs', 'rv', 'rv2',
                 'pitch_type', 'p_throws', 'stand', 'player_name']]

rhp = features.loc[features['p_throws'] == 'R']
print('RHP shape:', rhp.shape)
lhp = features.loc[features['p_throws'] == 'L']
print('LHP shape:', lhp.shape)
ff = features.loc[features['pitch_type'] == 'FF']
fc = features.loc[features['pitch_type'] == 'FC']
si = features.loc[features['pitch_type'] == 'SI']
sl = features.loc[features['pitch_type'] == 'SL']
cu = features.loc[features['pitch_type'] == 'CU']
kc = features.loc[features['pitch_type'] == 'KC']
ch = features.loc[features['pitch_type'] == 'CH']
fs = features.loc[features['pitch_type'] == 'FS']
ff_r = rhp.loc[rhp['pitch_type'] == 'FF']
ff_l = lhp.loc[lhp['pitch_type'] == 'FF']
fc_r = rhp.loc[rhp['pitch_type'] == 'FC']
fc_l = lhp.loc[lhp['pitch_type'] == 'FC']
si_r = rhp.loc[rhp['pitch_type'] == 'SI']
si_l = lhp.loc[lhp['pitch_type'] == 'SI']
sl_r = rhp.loc[rhp['pitch_type'] == 'SL']
sl_l = lhp.loc[lhp['pitch_type'] == 'SL']
cu_r = rhp.loc[rhp['pitch_type'] == 'CU']
cu_l = lhp.loc[lhp['pitch_type'] == 'CU']
ch_r = rhp.loc[rhp['pitch_type'] == 'CH']
ch_l = lhp.loc[lhp['pitch_type'] == 'CH']

RHP shape: (496493, 20)
LHP shape: (208903, 20)


In [8]:
data.lin_weight_above_avg.min(), data.lin_weight_above_avg.max()

(-0.25, 1.395)

In [9]:
data.lin_weight_above_outs.min(), data.lin_weight_above_outs.max()

(0.0, 1.645)

In [10]:
data.lin_weights_rel_outs.min(), data.lin_weights_rel_outs.max()

(-0.207, 1.154)

In [11]:
data.rv.min(), data.rv.max()

(-0.263, 0.211)

In [12]:
los = ff.loc[ff['player_name'] == 'Rodón, Carlos']
los.describe()

,velo,spin_rate,pfx_-x,pfx_z,release_extension,release_pos_x,release_pos_z,plate_x,plate_z,re24,lin_weight_above_avg,lin_weight_above_outs,lin_weights_rel_outs,lin_weights_not_rel_outs,rv,rv2
count,1302.000000,1302.000000,1302.000000,1302.000000,1302.000000,1302.000000,1302.000000,1302.000000,1302.000000,1302.000000,301.000000,301.000000,1302.000000,301.000000,1302.000000,1302.000000
mean,95.358909,2293.425499,-7.938802,17.551060,6.110753,1.989985,6.654754,0.070438,2.895300,-0.007415,-0.031116,0.218884,-0.005973,0.181136,-0.049393,-5.536636
std,2.148140,83.500502,1.896620,1.668841,0.171105,0.190637,0.075664,0.688491,0.777091,0.197360,0.397056,0.397056,0.158164,0.328535,0.091938,19.824813
min,86.700000,1635.000000,-14.880000,12.480000,5.600000,1.410000,6.230000,-1.960000,-0.340000,-0.898000,-0.250000,0.000000,-0.207000,0.000000,-0.263000,-47.000000
25%,94.000000,2244.000000,-9.240000,16.470000,6.000000,1.880000,6.600000,-0.400000,2.360000,0.000000,-0.250000,0.000000,0.000000,0.000000,-0.059000,-13.400000
50%,95.600000,2298.000000,-7.920000,17.520000,6.100000,1.990000,6.650000,0.085000,2.930000,0.000000,-0.250000,0.000000,0.000000,0.000000,-0.042000,-4.200000
75%,97.000000,2350.750000,-6.600000,18.600000,6.200000,2.110000,6.700000,0.560000,3.430000,0.000000,0.314000,0.564000,0.000000,0.467000,0.021000,2.800000
max,100.500000,2543.000000,-2.280000,22.680000,6.900000,2.590000,6.870000,1.880000,6.250000,2.044000,1.395000,1.645000,1.154000,1.361000,0.211000,111.800000


In [13]:
rob = ff.loc[ff['player_name'] == 'Ray, Robbie']
#RV / Tot. # of that Pitch Type * 100
lin = ((rob['lin_weights_rel_outs'] * 100) / 1864)
print(lin.mean())
rob.describe()

3.865308810256265e-05


,velo,spin_rate,pfx_-x,pfx_z,release_extension,release_pos_x,release_pos_z,plate_x,plate_z,re24,lin_weight_above_avg,lin_weight_above_outs,lin_weights_rel_outs,lin_weights_not_rel_outs,rv,rv2
count,1864.000000,1864.000000,1864.000000,1864.000000,1864.000000,1864.000000,1864.000000,1864.000000,1864.000000,1864.000000,471.000000,471.000000,1864.000000,471.000000,1864.000000,1864.000000
mean,94.776663,2272.281116,-8.184785,17.125107,6.475536,2.510799,6.052323,-0.118852,2.766395,-0.005027,0.003554,0.253554,0.000720,0.209809,-0.046915,-4.619474
std,1.246525,73.002477,1.958454,1.591146,0.181756,0.183995,0.110140,0.722570,0.713858,0.213637,0.444482,0.444482,0.184752,0.367748,0.087803,21.498173
min,90.600000,1493.000000,-13.560000,5.520000,5.800000,1.890000,5.470000,-2.330000,0.690000,-0.796000,-0.250000,0.000000,-0.207000,0.000000,-0.263000,-47.000000
25%,94.000000,2225.000000,-9.480000,16.080000,6.400000,2.390000,5.980000,-0.610000,2.270000,0.000000,-0.250000,0.000000,0.000000,0.000000,-0.057000,-13.400000
50%,94.800000,2272.000000,-8.220000,17.160000,6.500000,2.530000,6.050000,-0.130000,2.740000,0.000000,-0.250000,0.000000,0.000000,0.000000,-0.042000,-4.200000
75%,95.600000,2320.250000,-6.840000,18.120000,6.600000,2.640000,6.130000,0.372500,3.232500,0.000000,0.314000,0.564000,0.000000,0.467000,0.021000,2.800000
max,98.300000,2506.000000,-2.040000,21.960000,7.100000,3.140000,6.370000,2.650000,5.260000,2.370000,1.395000,1.645000,1.154000,1.361000,0.211000,111.800000


In [14]:
joe = ff.loc[ff['player_name'] == 'Kelly, Joe']
#RV / Tot. # of that Pitch Type * 100
rv = (joe['rv'] *100)
print(rv.mean())
joe.describe()

-4.214285714285715


,velo,spin_rate,pfx_-x,pfx_z,release_extension,release_pos_x,release_pos_z,plate_x,plate_z,re24,lin_weight_above_avg,lin_weight_above_outs,lin_weights_rel_outs,lin_weights_not_rel_outs,rv,rv2
count,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,10.000000,10.000000,49.000000,10.000000,49.000000,49.000000
mean,97.534694,2128.346939,11.079184,10.738776,6.024490,-1.701633,5.392245,-0.296531,2.900408,-0.051184,-0.108400,0.141600,-0.018327,0.117200,-0.042143,-6.046939
std,1.328933,139.445412,1.277823,3.431832,0.167743,0.134030,0.126380,0.785288,0.877916,0.232329,0.298519,0.298519,0.113065,0.247079,0.079207,16.387001
min,94.200000,1817.000000,7.440000,4.440000,5.500000,-2.000000,5.150000,-1.870000,0.750000,-0.789000,-0.250000,0.000000,-0.207000,0.000000,-0.263000,-39.400000
25%,96.600000,2011.000000,10.320000,8.160000,5.900000,-1.810000,5.290000,-0.830000,2.290000,0.000000,-0.250000,0.000000,0.000000,0.000000,-0.057000,-5.700000
50%,97.800000,2132.000000,11.520000,10.200000,6.000000,-1.670000,5.390000,-0.080000,2.910000,0.000000,-0.250000,0.000000,0.000000,0.000000,-0.042000,-3.600000
75%,98.400000,2236.000000,12.000000,13.680000,6.200000,-1.600000,5.490000,0.240000,3.490000,0.000000,-0.250000,0.000000,0.000000,0.000000,0.028000,2.800000
max,99.800000,2367.000000,13.080000,16.800000,6.300000,-1.450000,5.670000,1.070000,4.660000,0.578000,0.458000,0.708000,0.379000,0.586000,0.081000,34.300000


In [15]:
ty = ff.loc[ff['player_name'] == 'Clippard, Tyler']
rv = (ty['rv'] *100)
print(rv.mean())
ty.describe()

-4.97157894736842


,velo,spin_rate,pfx_-x,pfx_z,release_extension,release_pos_x,release_pos_z,plate_x,plate_z,re24,lin_weight_above_avg,lin_weight_above_outs,lin_weights_rel_outs,lin_weights_not_rel_outs,rv,rv2
count,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,20.000000,20.000000,95.000000,20.000000,95.000000,95.000000
mean,88.972632,2255.842105,5.813053,19.004211,6.155789,-1.250316,6.025895,0.321053,2.894211,-0.033000,-0.100200,0.149800,-0.017474,0.123950,-0.049716,-6.718947
std,0.606427,69.431171,1.580751,2.184545,0.111771,0.262092,0.127475,0.718289,0.793317,0.182036,0.316567,0.316567,0.122623,0.261897,0.093623,17.962356
min,87.400000,2052.000000,2.160000,7.800000,5.900000,-2.190000,5.680000,-1.890000,1.230000,-0.649000,-0.250000,0.000000,-0.207000,0.000000,-0.263000,-47.000000
25%,88.550000,2210.500000,4.620000,18.420000,6.100000,-1.385000,5.945000,-0.135000,2.335000,0.000000,-0.250000,0.000000,0.000000,0.000000,-0.059000,-18.700000
50%,89.000000,2260.000000,6.000000,19.440000,6.200000,-1.230000,6.030000,0.330000,2.870000,0.000000,-0.250000,0.000000,0.000000,0.000000,-0.042000,-4.200000
75%,89.500000,2300.000000,6.900000,20.160000,6.200000,-1.065000,6.120000,0.775000,3.415000,0.000000,-0.250000,0.000000,0.000000,0.000000,0.028000,2.800000
max,90.200000,2423.000000,9.840000,22.680000,6.400000,-0.620000,6.330000,2.080000,5.060000,0.966000,0.766000,1.016000,0.634000,0.840000,0.211000,58.900000


In [16]:
ian = ff.loc[ff['player_name'] == 'Kennedy, Ian']
ian.describe()

,velo,spin_rate,pfx_-x,pfx_z,release_extension,release_pos_x,release_pos_z,plate_x,plate_z,re24,lin_weight_above_avg,lin_weight_above_outs,lin_weights_rel_outs,lin_weights_not_rel_outs,rv,rv2
count,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.00000,179.000000,179.000000,740.000000,179.000000,740.000000,740.000000
mean,94.098108,2427.086486,10.783459,17.706162,6.505000,-1.616500,5.374649,-0.005932,2.860419,-0.00738,-0.024223,0.225777,-0.004872,0.186844,-0.045877,-5.074865
std,0.787072,97.200353,1.694850,1.346729,0.137302,0.126356,0.086982,0.742428,0.757489,0.21267,0.421560,0.421560,0.171425,0.348817,0.084355,20.162234
min,91.600000,1294.000000,5.760000,12.480000,6.100000,-2.070000,5.130000,-2.190000,0.680000,-0.78900,-0.250000,0.000000,-0.207000,0.000000,-0.263000,-47.000000
25%,93.600000,2373.000000,9.600000,16.920000,6.400000,-1.700000,5.310000,-0.492500,2.310000,0.00000,-0.250000,0.000000,0.000000,0.000000,-0.057000,-13.400000
50%,94.100000,2427.000000,10.680000,17.760000,6.500000,-1.610000,5.380000,0.010000,2.860000,0.00000,-0.250000,0.000000,0.000000,0.000000,-0.042000,-3.600000
75%,94.600000,2473.000000,11.910000,18.600000,6.600000,-1.527500,5.432500,0.452500,3.342500,0.00000,0.314000,0.564000,0.000000,0.467000,0.021000,2.800000
max,96.400000,3160.000000,16.320000,21.600000,7.000000,-1.180000,5.580000,2.140000,5.300000,1.87400,1.395000,1.645000,1.154000,1.361000,0.211000,111.800000


In [17]:
ff.groupby(['player_name']).mean()

,velo,spin_rate,pfx_-x,pfx_z,release_extension,release_pos_x,release_pos_z,plate_x,plate_z,re24,lin_weight_above_avg,lin_weight_above_outs,lin_weights_rel_outs,lin_weights_not_rel_outs,rv,rv2
player_name,,,,,,,,,,,,,,,,
"Abad, Fernando",91.688889,2112.111111,-5.340000,16.200000,5.855556,0.453333,6.175556,0.327778,2.856667,0.167833,0.523500,0.773500,0.144389,0.640000,-0.063500,8.088889
"Abbott, Cory",92.601449,2319.874396,9.578551,17.190725,6.188889,-2.643382,5.525990,0.279469,2.657150,0.039498,0.185500,0.435500,0.034097,0.360370,-0.027792,0.630435
"Abreu, Albert",97.786408,2132.349515,11.690097,15.424078,6.437864,-2.459126,5.657282,-0.403786,2.617573,0.004175,0.013875,0.263875,0.002660,0.218375,-0.043369,-4.070874
"Abreu, Bryan",95.660645,2214.764516,8.004000,17.052774,6.330645,-1.549839,5.957645,-0.009226,2.720226,0.009510,0.052909,0.302909,0.009310,0.250697,-0.030629,-2.131935
"Acevedo, Domingo",92.881818,2065.931818,12.231818,15.087273,5.757955,-1.639318,5.680114,0.169773,2.712841,0.036670,0.168900,0.418900,0.031750,0.346700,-0.029466,0.228409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Zimmermann, Jordan",89.714286,2391.571429,5.091429,16.542857,6.071429,-1.988571,5.722857,0.677143,2.508571,-0.032429,-0.250000,0.000000,-0.029571,0.000000,0.009000,-2.057143
"Zuber, Tyler",94.567577,2230.795222,9.014334,16.075904,5.870307,-1.641536,5.526621,0.075324,2.882218,0.028314,0.100905,0.350905,0.017945,0.290413,-0.026621,-0.867577
"de Geus, Brett",91.262500,2039.625000,9.120000,5.370000,5.687500,-1.562500,5.680000,0.261250,2.456250,-0.023125,-0.250000,0.000000,-0.051750,0.000000,-0.032500,-8.425000


In [18]:
data2 = data.copy()
data2.dropna(subset = ['events'], inplace = True)

#data2.to_csv('../data/model-plate-app-rv.csv')
print(data2.shape)
data2.head(10)

(180159, 68)


,player_name,p_throws,pitch_type,velo,spin_rate,spin_axis,pfx_-x,pfx_z,bauer_units,effective_speed,release_pos_-x,release_pos_x,release_pos_z,release_extension,release_pos_y,plate_-x,plate_x,plate_z,type,balls,strikes,pitch_count,delta_run_exp,stand,bb_type,description,events,hit_distance_sc,exit_velo,launch_angle,launch_speed_angle,woba_value,woba_denom,xba,xwobacon,babip_value,iso_value,at_bat_number,pitch_number,inning,inning_topbot,home_score,away_score,post_home_score,post_away_score,on_1b,on_2b,on_3b,outs_when_up,is_strike,is_ball,final_pitch_ab,out_to_end_inning,home_runs,away_runs,runs,re,re_change,re_end_state,re24,lin_weight_above_avg,lin_weight_above_outs,woba_scale,lin_weights_rel_outs,lin_weights_not_rel_outs,woba,rv,rv2
0,"Smith, Will",L,FF,92.3,2330.0,148.0,-8.28,16.56,25.243770,92.8,-1.40,1.40,6.80,6.5,54.03,0.69,-0.69,2.83,X,1,2,1-2,-0.073,R,ground_ball,hit_into_play,field_out,13.0,95.2,-13.0,2.0,0.0,1.0,0.174,0.158,0.0,0.0,61,4,9,0,5,0,5,0,0,0,0,2,1,0,1,1,0,0,0,0.098,-0.098,0.000,-0.098,-0.25,0.0,1.209,-0.207,0.0,0.223,-0.153,-36.0
4,"Smith, Will",L,FF,91.2,2281.0,143.0,-7.56,15.36,25.010965,90.9,-1.49,1.49,6.66,6.3,54.15,0.31,-0.31,2.80,X,1,0,1-0,-0.189,L,ground_ball,hit_into_play,field_out,9.0,93.3,-18.0,2.0,0.0,1.0,0.100,0.090,0.0,0.0,60,2,9,0,5,0,5,0,0,0,0,1,1,0,1,0,0,0,0,0.254,-0.156,0.098,-0.156,-0.25,0.0,1.209,-0.207,0.0,0.355,-0.042,-24.9
6,"Smith, Will",L,SL,80.3,2328.0,304.0,3.96,2.16,28.991283,80.8,-1.81,1.81,6.55,6.2,54.30,0.69,-0.69,0.83,S,0,2,0-2,-0.152,L,NaN,strike,field_out,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,0.0,0.0,59,3,9,0,5,0,5,0,0,0,0,0,1,0,1,0,0,0,0,0.481,-0.227,0.254,-0.227,-0.25,0.0,1.209,-0.207,0.0,0.196,-0.134,-34.1
9,"Gsellman, Robert",R,SI,94.3,1982.0,221.0,16.20,10.56,21.018028,94.4,1.20,-1.20,6.05,6.5,53.98,0.26,-0.26,1.78,X,0,2,0-2,-0.061,R,popup,hit_into_play,field_out,140.0,75.3,65.0,3.0,0.0,1.0,0.000,0.000,0.0,0.0,58,3,8,1,5,0,5,0,0,0,0,2,1,0,1,1,0,0,0,0.098,-0.098,0.000,-0.098,-0.25,0.0,1.209,-0.207,0.0,0.196,-0.134,-34.1
12,"Gsellman, Robert",R,SL,90.8,2052.0,215.0,0.36,5.04,22.599119,90.9,1.48,-1.48,5.95,6.4,54.14,-0.98,0.98,1.12,S,0,2,0-2,-0.101,R,NaN,strike,field_out,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,0.0,0.0,57,6,8,1,5,0,5,0,0,0,0,1,1,0,1,0,0,0,0,0.254,-0.156,0.098,-0.156,-0.25,0.0,1.209,-0.207,0.0,0.196,-0.134,-34.1
18,"Gsellman, Robert",R,SL,90.5,2133.0,216.0,-0.12,7.44,23.569061,91.1,1.51,-1.51,5.93,6.3,54.23,-0.01,0.01,2.23,X,1,2,1-2,-0.173,L,line_drive,hit_into_play,field_out,294.0,96.8,18.0,4.0,0.0,1.0,0.409,0.457,0.0,0.0,56,4,8,1,5,0,5,0,0,0,0,0,1,0,1,0,0,0,0,0.481,-0.227,0.254,-0.227,-0.25,0.0,1.209,-0.207,0.0,0.223,-0.153,-36.0
22,"Martin, Chris",R,CH,87.9,1160.0,234.0,7.68,1.80,13.196815,88.8,2.98,-2.98,6.23,6.7,53.81,0.36,-0.36,1.26,S,1,2,1-2,-0.073,L,NaN,strike,field_out,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,0.0,0.0,55,5,8,0,5,0,5,0,0,0,0,2,1,0,1,1,0,0,0,0.098,-0.098,0.000,-0.098,-0.25,0.0,1.209,-0.207,0.0,0.223,-0.153,-36.0
27,"Martin, Chris",R,FF,94.9,2224.0,216.0,9.00,14.16,23.435195,95.5,2.71,-2.71,6.38,6.6,53.86,0.10,-0.10,1.94,X,0,0,0-0,-0.163,R,fly_ball,hit_into_play,field_out,308.0,91.8,29.0,3.0,0.0,1.0,0.109,0.156,0.0,0.0,54,1,8,0,5,0,5,0,0,0,0,1,1,0,1,0,0,0,0,0.254,-0.156,0.098,-0.156,-0.25,0.0,1.209,-0.207,0.0,0.314,-0.036,-24.3
28,"Martin, Chris",R,FC,91.1,2156.0,177.0,-3.00,5.28,23.666301,92.1,2.85,-2.85,6.39,6.7,53.80,-0.67,0.67,2.62,X,1,0,1-0,-0.279,L,ground_ball,hit_into_play,field_out,6.0,76.1,-25.0,2.0,0.0,1.0,0.043,0.039,0.0,0.0,53,2,8,0,5,0,5,0,0,0,0,0,1,0,1,0,0,0,0,0.481,-0.227,0.254,-0.227,-0.25,0.0,1.209,-0.207,0.0,0.355,-0.042,-24.9
30,"Hembree, Heath",R,FF,93.5,2419.0,219.0,12.12,17.28,25.871658,93.9,0.45,-0.45,5.87,6.7,53.84,-0.91,0.91,2.50,X,0,2,0-2,-0.061,R,ground_ball,hit_into_play,field_out,7.0,99.7,-18.0,2.0,0.0,1.0,0.177,0.171,0.0,0.0,52,3,7,1,5,0,5,0,0,0,0,2,1,0,1,1,0,0,0,0.098,-0.098,0.000,-0.098,-0.25,0.0,1.209,-0.207,0.0,0.196,-0.134,-34.1
